In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("movie_dataset.csv")

In [3]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
features = ['keywords','cast','genres','director']

In [5]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

In [6]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string

In [7]:
df["combined_features"] = df.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column

In [8]:
df.iloc[0].combined_features

'culture clash future space war space colony society Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Action Adventure Fantasy Science Fiction James Cameron'

In [9]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object

In [10]:
cosine_sim = cosine_similarity(count_matrix)

In [11]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]
def get_index_from_genres(genres):
    return df[df.genres == genres]["index"].values[0]

In [12]:
def get_homepage_from_index(index):
    return df[df.index == index]["homepage"].values[0]
def get_director_from_index(index):
    return df[df.index == index]["director"].values[0]
def get_overview_from_index(index):
    return df[df.index == index]["overview"].values[0]
def get_vote_average_from_index(index):
    return df[df.index == index]["vote_average"].values[0]
def get_genres_from_index(index):
    return df[df.index == index]["genres"].values[0]
def get_cast_from_index(index):
    return df[df.index == index]["cast"].values[0]

In [20]:
ch =input("Would you to choose movie similar to another movie you watched or based on a genre? Select M/G: ")
rating= float(input(" Preferred minimum rating: "))

Would you to choose movie similar to another movie you watched or based on a genre? Select M/G: M
 Preferred minimum rating: 8


In [21]:
if(ch=='G'):
    genres_user_likes=input()
    movie_index = get_index_from_genres(genres_user_likes)
    similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
    i=1
    print("Top 5 movies with genre "+genres_user_likes+" are:\n")
    for element in sorted_similar_movies:
        if get_vote_average_from_index(element[0])>rating:
            print(" \033[1m","\033[4m",i,")", get_title_from_index(element[0])," \033[0m")
            print(" \033[1m"," Link :"," \033[0m",get_homepage_from_index(element[0]))
            print(" \033[1m"," Director :"," \033[0m",get_director_from_index(element[0]))
            print(" \033[1m"," Cast :"," \033[0m",get_cast_from_index(element[0]))
            print(" \033[1m"," Genres :"," \033[0m",get_genres_from_index(element[0]))
            print(" \033[1m"," Overview : "," \033[0m",get_overview_from_index(element[0]))
            print(" \033[1m"," Vote_Average : "," \033[0m",get_vote_average_from_index(element[0]))
            print()
            i=i+1
            if i>5:
                break
            
else:
    movie_user_likes = input()
    movie_index = get_index_from_title(movie_user_likes)
    similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
    i=1
    print("Top 5 similar movies to "+movie_user_likes+" are:\n")
    for element in sorted_similar_movies:
        if get_vote_average_from_index(element[0])>rating:
            print(" \033[1m","\033[4m",i,")", get_title_from_index(element[0])," \033[0m")
            print(" \033[1m"," Link :"," \033[0m",get_homepage_from_index(element[0]))
            print(" \033[1m"," Director :"," \033[0m",get_director_from_index(element[0]))
            print(" \033[1m"," Cast :"," \033[0m",get_cast_from_index(element[0]))
            print(" \033[1m"," Genres :"," \033[0m",get_genres_from_index(element[0]))
            print(" \033[1m"," Overview : "," \033[0m",get_overview_from_index(element[0]))
            print(" \033[1m"," Vote_Average : "," \033[0m",get_vote_average_from_index(element[0]))
            print()
            i=i+1
            if i>5:
                break

Toy Story
Top 5 similar movies to Toy Story are:

   1 ) Dancer, Texas Pop. 81  
   Link :   nan
   Director :   Tim McCanlies
   Cast :   Breckin Meyer Peter Facinelli Eddie Mills Ethan Embry Ashley Johnson
   Genres :   Comedy Drama Family
   Overview :    Four guys, best friends, have grown up together in DANCER, TEXAS POP. 81, a tiny town in West Texas. Years ago, they made a solemn vow to leave town together as soon as they graduate. Now, it's that weekend and the time has come to "put up or shut up." The clock is ticking and as all 81 people in the town watch, comment, offer advice and place bets, these four very different boys with unique backgrounds struggle with the biggest decision of their lives... whether to stay or leave home.
   Vote_Average :    10.0

   2 ) Forrest Gump  
   Link :   nan
   Director :   Robert Zemeckis
   Cast :   Tom Hanks Robin Wright Gary Sinise Mykelti Williamson Sally Field
   Genres :   Comedy Drama Romance
   Overview :    A man with a low IQ has